## Analyzing a Movie Dataset

### Getting general information about the dataset

In [123]:
# Loading the dataset
import pandas as pd
import numpy as np

movies_df = pd.read_csv("imdb_top_1000.csv")

# Print first few rows of the dataset
movies_df.head()



,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [124]:
# find the row by column
movies_df.shape

(1000, 16)

In [125]:
movies_df.size

16000

In [126]:
# Summary statistics of numerical columns
movies_df.describe()
# for this dataset, it doesn't tell me too much

,IMDB_Rating,Meta_score,No_of_Votes
count,1000.000000,843.000000,1.000000e+03
mean,7.949300,77.971530,2.736929e+05
std,0.275491,12.376099,3.273727e+05
min,7.600000,28.000000,2.508800e+04
25%,7.700000,70.000000,5.552625e+04
50%,7.900000,79.000000,1.385485e+05
75%,8.100000,87.000000,3.741612e+05
max,9.300000,100.000000,2.343110e+06


In [127]:
# Get general information about the dataframe
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


In [128]:
movies_df.set_index('Series_Title', inplace=True) # Change index to be Title column
movies_df.head(4)

,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Series_Title,,,,,,,,,,,,,,,
The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"


In [129]:
movies_df.tail(10)

,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Series_Title,,,,,,,,,,,,,,,
Giù la testa,https://m.media-amazon.com/images/M/MV5BYjRmY2...,1971,PG,157 min,"Drama, War, Western",7.6,A low-life bandit and an I.R.A. explosives exp...,77.0,Sergio Leone,Rod Steiger,James Coburn,Romolo Valli,Maria Monti,30144,"696,690"
Kelly's Heroes,https://m.media-amazon.com/images/M/MV5BMzAyND...,1970,GP,144 min,"Adventure, Comedy, War",7.6,A group of U.S. soldiers sneaks across enemy l...,50.0,Brian G. Hutton,Clint Eastwood,Telly Savalas,Don Rickles,Carroll O'Connor,45338,"1,378,435"
The Jungle Book,https://m.media-amazon.com/images/M/MV5BMjAwMT...,1967,U,78 min,"Animation, Adventure, Family",7.6,Bagheera the Panther and Baloo the Bear have a...,65.0,Wolfgang Reitherman,Phil Harris,Sebastian Cabot,Louis Prima,Bruce Reitherman,166409,"141,843,612"
Blowup,https://m.media-amazon.com/images/M/MV5BYTE4YW...,1966,A,111 min,"Drama, Mystery, Thriller",7.6,A fashion photographer unknowingly captures a ...,82.0,Michelangelo Antonioni,David Hemmings,Vanessa Redgrave,Sarah Miles,John Castle,56513,NaN
A Hard Day's Night,https://m.media-amazon.com/images/M/MV5BZjQyMG...,1964,U,87 min,"Comedy, Music, Musical",7.6,"Over two ""typical"" days in the life of The Bea...",96.0,Richard Lester,John Lennon,Paul McCartney,George Harrison,Ringo Starr,40351,"13,780,024"
Breakfast at Tiffany's,https://m.media-amazon.com/images/M/MV5BNGEwMT...,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
Giant,https://m.media-amazon.com/images/M/MV5BODk3Yj...,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
From Here to Eternity,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
Lifeboat,https://m.media-amazon.com/images/M/MV5BZTBmMj...,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


### Cleaning up the dataset
* adding columns before analysis
* removing rows
* cleaning up strings
* converting data types

In [130]:
# Add a 'Rank' column
movies_df['Rank'] = range(1, len(movies_df) + 1)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, The Shawshank Redemption to The 39 Steps
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Certificate    899 non-null    object 
 3   Runtime        1000 non-null   object 
 4   Genre          1000 non-null   object 
 5   IMDB_Rating    1000 non-null   float64
 6   Overview       1000 non-null   object 
 7   Meta_score     843 non-null    float64
 8   Director       1000 non-null   object 
 9   Star1          1000 non-null   object 
 10  Star2          1000 non-null   object 
 11  Star3          1000 non-null   object 
 12  Star4          1000 non-null   object 
 13  No_of_Votes    1000 non-null   int64  
 14  Gross          831 non-null    object 
 15  Rank           1000 non-null   int64  
dtypes: float64(2), int64(2), object(12)
memory usage: 132.8+ KB


In [131]:
# get list of column names
new_columns = list(movies_df.columns)
# remove the 'Rank' column from the list
new_columns.remove('Rank')
# insert 'Rank' column at specified index
new_columns.insert(0, 'Rank')
# Reorder the DataFrame Columns
movies_df = movies_df[new_columns]
# Check DataFrame info
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, The Shawshank Redemption to The 39 Steps
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           1000 non-null   int64  
 1   Poster_Link    1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 132.8+ KB


In [132]:
# Sorting the data by ratings
movies_df = movies_df.sort_values(by='IMDB_Rating', ascending=False)

movies_df.reset_index(inplace=True)

movies_df.set_index('Rank', inplace=True)

movies_df

,Series_Title,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,,,
1,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
2,The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
3,The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
4,The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
5,12 Angry Men,https://m.media-amazon.com/images/M/MV5BMWU4N2...,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,Zombieland,https://m.media-amazon.com/images/M/MV5BMTU5MD...,2009,A,88 min,"Adventure, Comedy, Fantasy",7.6,A shy student trying to reach his family in Oh...,73.0,Ruben Fleischer,Jesse Eisenberg,Emma Stone,Woody Harrelson,Abigail Breslin,520041,"75,590,286"
912,La piel que habito,https://m.media-amazon.com/images/M/MV5BYmFmNj...,2011,R,120 min,"Drama, Horror, Thriller",7.6,"A brilliant plastic surgeon, haunted by past t...",70.0,Pedro Almodóvar,Antonio Banderas,Elena Anaya,Jan Cornet,Marisa Paredes,138959,"3,185,812"
911,Moneyball,https://m.media-amazon.com/images/M/MV5BMjAxOT...,2011,PG-13,133 min,"Biography, Drama, Sport",7.6,Oakland A's general manager Billy Beane's succ...,87.0,Bennett Miller,Brad Pitt,Robin Wright,Jonah Hill,Philip Seymour Hoffman,369529,"75,605,492"


In [133]:
movies_df.sort_index().head()

,Series_Title,Poster_Link,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,,,
1,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
2,The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
3,The Dark Knight,https://m.media-amazon.com/images/M/MV5BMTMxNT...,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
4,The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
5,12 Angry Men,https://m.media-amazon.com/images/M/MV5BMWU4N2...,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [134]:
# Handling missing data
print(movies_df.isnull().sum())

Series_Title       0
Poster_Link        0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64


In [135]:
# drop unused columns
movies_df.drop(['Certificate', 'Poster_Link'], axis=1, inplace=True)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Runtime        1000 non-null   object 
 3   Genre          1000 non-null   object 
 4   IMDB_Rating    1000 non-null   float64
 5   Overview       1000 non-null   object 
 6   Meta_score     843 non-null    float64
 7   Director       1000 non-null   object 
 8   Star1          1000 non-null   object 
 9   Star2          1000 non-null   object 
 10  Star3          1000 non-null   object 
 11  Star4          1000 non-null   object 
 12  No_of_Votes    1000 non-null   int64  
 13  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(11)
memory usage: 117.2+ KB


In [136]:
#movies_df.fillna(0)
#print(movies_df.isna().sum())
#lets remove rows with null data
movies_df.dropna(inplace=True)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750 entries, 1 to 911
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   750 non-null    object 
 1   Released_Year  750 non-null    object 
 2   Runtime        750 non-null    object 
 3   Genre          750 non-null    object 
 4   IMDB_Rating    750 non-null    float64
 5   Overview       750 non-null    object 
 6   Meta_score     750 non-null    float64
 7   Director       750 non-null    object 
 8   Star1          750 non-null    object 
 9   Star2          750 non-null    object 
 10  Star3          750 non-null    object 
 11  Star4          750 non-null    object 
 12  No_of_Votes    750 non-null    int64  
 13  Gross          750 non-null    object 
dtypes: float64(2), int64(1), object(11)
memory usage: 87.9+ KB


In [137]:
# determine if there is any duplicated data
sum(movies_df.duplicated())

0

In [138]:
# clean up the strings in Gross column for converting to numerical values
movies_df['Gross'] = movies_df['Gross'].replace(r'[\$,]', '', regex=True).str.strip()
display(movies_df.head(3)) # check that Gross column has changed
movies_df['Gross'] = pd.to_numeric(movies_df['Gross'])
movies_df.info()


,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,
1,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469
2,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411
3,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444


<class 'pandas.core.frame.DataFrame'>
Index: 750 entries, 1 to 911
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   750 non-null    object 
 1   Released_Year  750 non-null    object 
 2   Runtime        750 non-null    object 
 3   Genre          750 non-null    object 
 4   IMDB_Rating    750 non-null    float64
 5   Overview       750 non-null    object 
 6   Meta_score     750 non-null    float64
 7   Director       750 non-null    object 
 8   Star1          750 non-null    object 
 9   Star2          750 non-null    object 
 10  Star3          750 non-null    object 
 11  Star4          750 non-null    object 
 12  No_of_Votes    750 non-null    int64  
 13  Gross          750 non-null    int64  
dtypes: float64(2), int64(2), object(10)
memory usage: 87.9+ KB


In [139]:
movies_df['Runtime'] = movies_df['Runtime'].str.replace(' min', '')
# Convert the 'Runtime' column to numeric type
movies_df['Runtime'] = pd.to_numeric(movies_df['Runtime'])
display(movies_df.head())
print(f"Data type for 'Runtime' is: {movies_df['Runtime'].dtype}") # checking runtime datatype

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
Rank,,,,,,,,,,,,,,
1,The Shawshank Redemption,1994,142,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469
2,The Godfather,1972,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411
3,The Dark Knight,2008,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444
4,The Godfather: Part II,1974,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000
5,12 Angry Men,1957,96,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000


Data type for 'Runtime' is: int64


In [ ]:
# drop rows with missing gross data
movies_df.dropna(subset=['Gross'], inplace=True)
movies_df

In [ ]:
movies_df['Released_Year'] = pd.to_datetime(movies_df['Released_Year'])
movies_df.head()
# doesnt work because dont have the full date only the year

In [ ]:
# Create a mask to identify rows with valid years (to remove rows with invalid year values)
valid_year_mask = movies_df['Released_Year'].str.match(r'^\d{4}$')

# Filter out rows with invalid years
movies_df = movies_df[valid_year_mask]

# Convert 'Released_Year' to datetime
movies_df['Released_Year'] = pd.to_datetime(movies_df['Released_Year'] + '-01-01')

# Print the updated DataFrame
movies_df.head()


In [ ]:
movies_df['Released_Year'] = pd.to_datetime(movies_df['Released_Year'])
movies_df['Released_Year'] = movies_df['Released_Year'].dt.year

In [ ]:
# String cleaning
movies_df['Genre'] = movies_df['Genre'].str.strip().str.lower()
print(movies_df['Genre'].head())

In [ ]:
# correcting data types
movies_df['Gross'] = movies_df['Gross'].str.replace(',', '').astype(float)
movies_df

### Analyzing the data

In [ ]:
# Get genre count of top 5 most common genre
movies_df['Genre'].value_counts().head(5)

In [ ]:
# Show only movies with genre name Drama
movies_df[movies_df['Genre'] == 'Drama']

In [ ]:
# Get movies with released after 2015
cutoff_date = pd.to_datetime('2015-01-01')
after_year = movies_df['Released_Year'] > cutoff_date
movies_df[after_year].head(5) #display only 5 rows

In [ ]:
#movie with highest votes
# Sort the DataFrame by 'Number of Votes' column in descending order
movies_sorted_by_votes = movies_df_clean.sort_values(by='No_of_Votes', ascending=False)

# Get the top 5 movies with the most votes
top_movies_by_votes = movies_sorted_by_votes.head(5)

# Print the top movies with the most votes
print("Top 5 movies with the most votes:")
display(top_movies_by_votes)



In [ ]:
#which movies have the longest runtime
# Find the movie with the longest runtime
movie_longest_runtime = movies_df_clean.nlargest(1, 'Runtime')

# Print the movie details
print("Movie with the longest runtime:")
movie_longest_runtime


In [ ]:
#movie with shortest runtime
# Find the movie with the shortest runtime
movie_shortest_runtime = movies_df_clean.nsmallest(1, 'Runtime')

# Print the movie details
print("Movie with the shortest runtime:")
movie_shortest_runtime


In [ ]:
# determine which year had the most released movies
best_year = movies_df.groupby('Released_Year').count()['Series_Title']
#best_year.shape
pd.set_option('display.max_rows', None)
best_year
#pd.reset_option('diplay.max_rows')

In [ ]:
# which movies had the hishgest gross
max_gross_index = movies_df['Gross'].idxmax() #get index of movie with highest gross
movie_with_highest_gross = movies_df.loc[max_gross_index, 'Series_Title']
print(movie_with_highest_gross)

In [ ]:
# Grouping the movies and looking at 
genre_group = movies_df.groupby('Genre').agg({'IMDB_Rating': 'mean', 'Gross': 'sum'})
print(genre_group)

### Merging Files example

In [ ]:
# loading new datasets
df1 = pd.read_csv("tmdb_5000_credits.csv")
df2 = pd.read_csv("tmdb_5000_movies.csv")
df1.info()

In [ ]:
df2.info()

In [ ]:
df1.columns = ['id', 'title', 'cast', 'crew']
df2 = df2.merge(df1, on='id')
df2.info()

In [ ]:
# Look at the new data
df2.head(5)

In [ ]:
# look at mean voting average
c = df2['vote_average'].mean()
c
#so most movies in this list is approx a 6 out of 10 rating

In [ ]:
# lets get only movies that have a certain number of votes so movies with only 3 votes aren't included
m = df2['vote_count'].quantile(0.9)
qualify_movies = df2.copy().loc[df2['vote_count'] >= m] # make a new DataFrame
qualify_movies.shape #481 movies out of the 5000 movies qualify

In [ ]:
def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * c)

qualify_movies['score'] = qualify_movies.apply(weighted_rating, axis=1)
qualify_movies = qualify_movies.sort_values('score', ascending=False)
qualify_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)